# Robustbench Testing
```shell
pip install git+https://github.com/RobustBench/robustbench.git
```

In [1]:
from robustbench.utils import load_model
from robustbench.model_zoo.enums import ThreatModel

base_model = load_model(model_name='Standard', model_dir='download/robustbench/', dataset='cifar10', threat_model=ThreatModel.corruptions)
base_model.eval()


ConnectionError: HTTPSConnectionPool(host='drive.google.com', port=443): Max retries exceeded with url: /uc?id=1t98aEuzeTL8P7Kpd5DIrCoCL21BNZUhC (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f020695c040>: Failed to establish a new connection: [Errno 101] Network is unreachable'))